In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [35]:
#doginfo.csv파일 데이터를 pandas를 이용해 읽어옵니다.
dog_data=pd.read_csv("doginfo2.csv")
dog_train=pd.read_csv("testDog2.csv")
kindCd=pd.read_csv("kindCd.csv")

dog_data = dog_data.dropna(axis=0)
dog_train = dog_train.dropna(axis=0)
kindCd_data = kindCd.dropna(axis=0)

print(dog_data)
print(dog_train)
print(kindCd_data)

      neuterYn sexCd  weight  noticeDays  age2  processState
0            N     F    7.46          10    12             0
1            N     M    7.00          14     1             1
2            U     M    4.50          11     2             0
3            N     M   10.00           8     1             0
4            N     M    6.00           8     4             0
...        ...   ...     ...         ...   ...           ...
22787        N     M    1.00          10     0             1
22788        N     M    1.00          10     0             1
22789        N     M    1.00          10     0             1
22790        U     F    6.00          12     3             0
22791        N     M    3.50          10     0             0

[22792 rows x 6 columns]
    neuterYn sexCd  weight  noticeDays  age2  processState
0          N     M     0.8          10     0             1
1          N     M     5.0          10     1             0
2          N     F     5.0          10    13             0
3     

In [36]:
# kindCd = np.array(kindCd_data, dtype = np.float64)
# kindCd_train=np.array(kindCd_data, dtype=np.float64)

# kindCd = kindCd.reshape(177)
# kindCd_train=kindCd_train.reshape(177)

# print(kindCd)
# print(kindCd.shape)

In [37]:
# # kindNum을 원핫 인코딩
# kindCd = pd.concat((pd.get_dummies(dog_data.kindNum, columns=kindCd), pd.DataFrame(columns=kindCd))).fillna(0)
# # kindCd

# # 학습데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
# dog_data.drop(['kindNum'], axis='columns', inplace=True)
# dog_data = pd.concat([dog_data, kindCd], axis=1)


# # kindNum을 원핫 인코딩
# kindCd_train = pd.concat((pd.get_dummies(dog_train.kindNum, columns=kindCd_train), pd.DataFrame(columns=kindCd_train))).fillna(0)

# # 테스트데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
# dog_train.drop(['kindNum'], axis='columns', inplace=True)
# dog_train = pd.concat([dog_train, kindCd_train], axis=1)

# print(dog_data)
# print(dog_train)

In [38]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis = 1)
        data = data.drop(column, axis = 1)
    return data

In [39]:
dummy_columns = ["neuterYn", "sexCd"]
data = dummy_data(dog_data, dummy_columns)
train_data = dummy_data(dog_train, dummy_columns)

print(data.head())
print(train_data.head())

data = np.array(data, dtype = np.float64)
train_data = np.array(train_data, dtype = np.float64)

   weight  noticeDays  age2  processState  neuterYn_N  neuterYn_U  neuterYn_Y  \
0    7.46          10    12             0           1           0           0   
1    7.00          14     1             1           1           0           0   
2    4.50          11     2             0           0           1           0   
3   10.00           8     1             0           1           0           0   
4    6.00           8     4             0           1           0           0   

   sexCd_F  sexCd_M  sexCd_Q  
0        1        0        0  
1        0        1        0  
2        0        1        0  
3        0        1        0  
4        0        1        0  
   weight  noticeDays  age2  processState  neuterYn_N  neuterYn_U  neuterYn_Y  \
0     0.8          10     0             1           1           0           0   
1     5.0          10     1             0           1           0           0   
2     5.0          10    13             0           1           0           0   
3  

In [40]:
a = data[:, :3]
b = data[:, 4:]

#numpy 배열에서 데이터 변화요인(kindCd, neuterYn, sexCd, weight, noticeDays, age2)으로 사용할 데이터를 뽑아냅니다.
xData = np.concatenate([a, b], axis = 1)

a=train_data[:, :3]
b=train_data[:, 4:]

testX=np.concatenate([a, b], axis = 1)

In [41]:
#numpy배열에서 결과(입양여부)로 사용할 데이터를 뽑아냅니다.
yData=data[:,[3]]
testY=train_data[:,[3]]

In [42]:
def model_performance(y_test, y_pred):    
    print('confusion matrix')
    print(metrics.confusion_matrix(y_test, y_pred))
    print('accuracy : {}'.format(metrics.accuracy_score(y_test, y_pred)))
    print('precision : {}'.format(metrics.precision_score(y_test, y_pred)))
    print('recall : {}'.format(metrics.recall_score(y_test, y_pred)))
    print('F1 : {}'.format(metrics.f1_score(y_test, y_pred)))

In [43]:
from sklearn.model_selection import GridSearchCV
params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 3, n_jobs = -1, verbose = 1)
grid_cv.fit(xData, yData)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   40.4s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:   47.0s finished
C:\Users\yunji\anaconda3\envs\dbswls\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


최적 하이퍼 파라미터:  {'max_depth': 12, 'min_samples_leaf': 18, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도: 0.6292


In [33]:
model = RandomForestClassifier(n_estimators = 100, 
                                max_depth = 12,
                                min_samples_leaf = 8,
                                min_samples_split = 8,
                                random_state = 0,
                                n_jobs = -1)
rf_model=model.fit(xData, yData)
y_pred = rf_model.predict(testX)
model_performance(testY, y_pred)
# len(y_pred)

print('총 테스트 개수 : %d, 오류 개수 : %d' %(len(testY), (testY != y_pred).sum()))
print('정확도 : %.2f' %metrics.accuracy_score(testY, y_pred))

C:\Users\yunji\anaconda3\envs\dbswls\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


confusion matrix
[[189 119]
 [113 189]]
accuracy : 0.6196721311475409
precision : 0.6136363636363636
recall : 0.6258278145695364
F1 : 0.6196721311475409
총 테스트 개수 : 610, 오류 개수 : 186068
정확도 : 0.62


In [80]:
rf_model.predict_proba(testX[:6])

array([[0.        , 1.        ],
       [0.64494874, 0.35505126],
       [0.5       , 0.5       ],
       [0.44796212, 0.55203788],
       [0.90261905, 0.09738095],
       [0.90261905, 0.09738095]])

In [81]:
joblib.dump(rf_model, 'randomforest.pkl')

['randomforest.pkl']